# Topic 14.6 - 金融计算器程序升级 - 版本 v2

本章我们将继续完善金融计算器程序，增加更多的功能和更好的用户体验。

- 当然，金融计算器这个项目其实也不算是一个特别复杂的项目，毕竟它只是一个命令行程序
- 因此，本章我们就对金融计算器进行一些简单的功能的改进，重点是让大家体验一下如何对一个已有的项目进行升级和改进
- 如果大家有兴趣的话，也可以尝试自己动手对这个项目进行更多的改进和完善

在学习本章之前，大家先拷贝一份 `financial_calculator_v1` 文件夹，并将其重命名为 `financial_calculator_v2`

- 我们本章主要在 `financial_calculator_v2` 文件夹中进行改进
- 并且大家记着将 `config.py` 与 `config_test.py` 中的路径前缀也进行相应的修改

## 1. 改进用户现金流和金钱时间价值的用户输入

在上一个版本中，我们让用户输入现金流和金钱时间价值的交互方式其实比较麻烦：

- 我们让用户输入现金流的方式是，用户需要按照提示依次输入每笔现金流的时间点和金额，如果有10笔现金流，就需要输入10 x 2 = 20 次输入
- 我们让用户输入金钱时间价值的方式是，依次输入 PV、FV、PMT、R、N 这5个参数，需要输入5次

这种交互方式其实并不友好，整个过程会比较繁琐，因此我们的改进想法是：

- 输入现金流时，直接输入现金流列表就行，例如 `-1000, 300, 400, 500, 600`，然后我们将这个字符串解析成现金流列表
- 输入金钱时间价值时，直接输入参数列表就行，例如 `-1000, None, -60, 0.05, 12`，就表示 PV=-1000, FV=None, PMT=-60, R=0.05, N=12，然后我们将这个字符串解析成参数元组


### (1) 改进现金流的用户输入

改进现金流的用户输入，涉及到接受用户输入的字符串，然后将其解析成现金流列表，在这个过程中有任何错误都需要提示用户重新输入。

这个过程的基本思路是：

- 首先，我们来确定一下合法的现金流字符串格式，就是以逗号分隔的数字列表，例如 `-1000, 300, 400, 500, 600`。

- 接下来，我们思考一下如何将这个字符串解析成现金流列表：

    - 首先，我们可以使用字符串的 `split(',')` 方法将字符串按逗号分隔成一个字符串列表，例如 `['-1000', ' 300', ' 400', ' 500', ' 600']`
    - 然后，列表中的每个字符串都需要去掉前后的空格，可以使用字符串的 `strip()` 方法，新的字符串列表就是 `['-1000', '300', '400', '500', '600']`
    - 接着，我们可以使用列表推导式将字符串列表中的每个字符串转换成浮点数，生成现金流列表，最终结果就是 `[-1000.0, 300.0, 400.0, 500.0, 600.0]`

- 在转换过程中，发生任何错误，例如输入的字符串中包含非数字字符，我们都需要捕获异常并提示用户重新输入

下面是实现这个功能的代码示例：

```python
def get_cash_flow_list():
    while True:
        print("-" * 40)
        print("请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：")
        cash_flow_str = input("请输入现金流列表：")
        try:
            cash_flow_list = []
            for cash in cash_flow_str.split(','):
                cash_flow_list.append(float(cash.strip()))
            print("现金流列表输入成功：", cash_flow_list)
            print("-" * 40)
            return cash_flow_list
        except ValueError:
            print("输入格式错误，请重新输入合法的现金流列表。")
```

如果使用列表推导式，会更简洁一些，代码如下：

```python
def get_cash_flow_list():
    while True:
        print("-" * 40)
        print("请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：")
        cash_flow_str = input("请输入现金流列表：")
        try:
            cash_flow_list = [float(cash.strip()) for cash in cash_flow_str.split(',')]
            print("现金流列表输入成功：", cash_flow_list)
            print("-" * 40)
            return cash_flow_list
        except ValueError:
            print("输入格式错误，请重新输入合法的现金流列表。")
```


更新了这个函数后，我们重新运行一下现金流功能的主函数，看看效果如何：

```text
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：1
----------------------------------------
----------------------------------------
请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：
请输入现金流列表：-1000,  w, 300, 400
输入格式错误，请重新输入合法的现金流列表。
----------------------------------------
请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：
请输入现金流列表：-100, 300, 400, 500, 600
现金流列表输入成功： [-100.0, 300.0, 400.0, 500.0, 600.0]
----------------------------------------
该现金流的内部收益率（IRR）为： [3.282899496427457]
请输入折现率（小数形式，如 0.05 表示 5%）：0.08
该现金流的净现值（NPV）为： 1358.6473
----------------------------------------
是否继续计算？(y/n)：n
----------------------------------------
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：q
----------------------------------------
```

我们可以看到，缩减了用户输入现金流的步骤之后，用户只需要输入一次字符串就可以了，大大简化了交互过程。

### (2) 改进金钱时间价值的用户输入

同样的道理，我们也可以改进金钱时间价值的用户输入方式，让用户直接输入参数列表字符串，例如 `-1000, None, -60, 0.05, 12`，然后我们将其解析成参数元组。

思路是和现金流列表类似的：

- 首先，我们使用字符串的 `split(',')` 方法将字符串按逗号分隔成一个字符串列表，例如 `['-1000', ' None', ' -60', ' 0.05', ' 12']`
- 然后，列表中的每个字符串都需要去掉前后的空格，可以使用字符串的 `strip()` 方法，新的字符串列表就是 `['-1000', 'None', '-60', '0.05', '12']`
- 接着，我们可以使用列表推导式将字符串列表中的每个字符串转换成对应的参数值：

    - 如果字符串是 `'None'`，则转换成 `None`
    - 否则，转换成浮点数

- 并且我们要加入输入判断，确保用户输入了5个参数，且有且仅有一个参数是 `None`，否则提示用户重新输入
- 其他任何错误，例如输入的字符串中包含非数字字符，我们都需要捕获异常并提示用户重新输入

根据这个思路，下面是实现这个功能的代码示例： 

```python
def get_time_value_inputs():
    while True:
        print("-" * 40)
        print("请输入金钱时间价值的参数：")
        print("格式为：PV, FV, PMT, R, N，需要求的参数请输入 None")
        print("例如：-1000, None, -60, 0.05, 12")
        params_str = input("请输入参数：")
        try:
            params_list = []
            for param in params_str.split(','):
                param = param.strip()
                if param == 'None':
                    params_list.append(None)
                else:
                    params_list.append(float(param))
            if len(params_list) != 5:
                print("输入参数数量错误，请输入5个参数。")
                continue
            if params_list.count(None) != 1:
                print("请输入且仅输入一个参数为 None。")
                continue
            if params_list[3] is not None and not (params_list[3] > -1):
                print("折现率 R 必须大于-1。")
                continue
            if params_list[4] is not None and not (params_list[4] > 0):
                print("期数 N 必须是正数。")
                continue
            print("金钱时间价值参数输入成功：", params_list)
            print("-" * 40)
            return tuple(params_list)
        except ValueError:
            print("输入格式错误，请重新输入合法的参数列表。")
```

我们运行一下金钱时间价值功能的主函数，看看效果如何：

```text
----------------------------------------
金钱时间价值计算功能：
0. 查看功能说明
1. 计算金钱时间价值
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：1
----------------------------------------
----------------------------------------
请输入金钱时间价值的参数：
格式为：PV, FV, PMT, R, N，需要求的参数请输入 None
例如：-1000, None, -60, 0.05, 12
请输入参数：None, 2000, 60, 0.05, 20
金钱时间价值参数输入成功： [None, 2000.0, 60.0, 0.05, 20.0]
----------------------------------------
参数输入为：
PV： None
FV： 2000.0
PMT： 60.0
R： 0.05
N： 20.0
****************************************
计算结果为：
PV： -1501.5115862984005
FV： 2000.0
PMT： 60.0
R： 0.05
N： 20.0
****************************************
输入任意内容返回主菜单：
----------------------------------------
----------------------------------------
金钱时间价值计算功能：
0. 查看功能说明
1. 计算金钱时间价值
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：q
----------------------------------------
```

同样，我们可以看到，缩减了用户输入金钱时间价值参数的步骤之后，用户只需要输入一次字符串就可以了，大大简化了交互过程。

## 2. 强化 IRR 计算功能

### (1) 实现求多个解的 IRR 函数

之前我们说，要强化一版 IRR 计算功能，让它支持多个解的输出，我们现在就来实现一下。

这里我们会用到一些数学知识，大家能听懂就听，听不懂就当做是一个别人帮忙实现好的函数直接来用就行了。

假设我们有一组现金流：`[-100, 80, -30, 90, -20]`

- 求解 IRR 可以写成一个多项式方程：

$$-100 + \frac{80}{1+IRR} + \frac{-30}{(1+IRR)^2} + \frac{90}{(1+IRR)^3} + \frac{-20}{(1+IRR)^4} = 0$$

- 令 $x = 1 + IRR$，则上式可以改写为：

$$
-100 + \frac{80}{x} + \frac{-30}{x^2} + \frac{90}{x^3} + \frac{-20}{x^4} = 0
$$

- 等式两边同时乘以 $x^4$，消去分母：

$$-100x^4 + 80x^3 - 30x^2 + 90x - 20 = 0$$

- 写成这个版本就是一个典型的多项式方程了

求解多项式方程的根，可以使用 NumPy 库中的 `numpy.roots()` 函数，它可以接受一个多项式的系数列表作为输入，返回该多项式的所有根

- 当然，这些根可能是实数根，也可能是复数根
- 我们只要实数根，所以要过滤掉复数根
- 而且，就算是实数根，它也写成了复数形式，例如 `2+0j`，我们也需要把它转换成浮点数形式 `2.0`

根据这个思路，我们的 `calculate_irr` 函数可以改写成下面这样：

```python
import numpy as np

def calculate_irr(cashflows):

    # 求所有根（x=1+r）
    roots = np.roots(cashflows)

    # 只保留实数根的实部
    roots_filtered = [r.real for r in roots if abs(r.imag) < 1e-8]

    # 转换为 irr，且只保留大于 -1 的解
    irr_values = [r - 1 for r in roots_filtered if r - 1 > -1]

    # 如果没有实数根，返回 None
    if not irr_values:
        return [None]
    # 如果有实数根，则按从小到大排序返回
    else:
        return sorted(irr_values)
```

### (2) 测试新的 IRR 函数

我们将这个函数替换原来的计算 IRR 的函数，然后重新跑一下 IRR 的测试函数，全部通过则说明函数正确：

```python
def test_calculate_irr_npv():

    def test_irr(cash_flows):
        
        irr_list = calculate_irr(cash_flows)
        irr_valid_list = [False for irr in irr_list]
        
        if irr_list == [None]:
            # 直接在这里判断现金流是否发生过变号（忽略 0）
            signs = [cf > 0 for cf in cash_flows if cf != 0]

            # 若列表非空且存在相邻符号不同，则说明发生变号
            if signs and any(sign != signs[0] for sign in signs):
                return [False] # 有变号（理论上可能有IRR）
            else:
                return [True]  # 无变号（理论上无IRR）

        else:
            for idx, irr in enumerate(irr_list):
                npv = calculate_npv(cash_flows, irr)
                if abs(npv) < 0.01:
                    irr_valid_list[idx] = True
                else:
                    irr_valid_list[idx] = False
        
        if all(irr_valid_list):
            return True
        else:
            return False

    def test_npv(cash_flows, discount_rate, npv_expected):
        npv_calculated = calculate_npv(cash_flows, discount_rate)
        if abs(npv_calculated - npv_expected) < 0.01:
            return True
        else:
            return False


    # 测试用例：格式为：(现金流列表, 折现率, NPV)
    test_cases = [
        ([-1000, 200, 300, 400, 500, 600], 0.08, 535.7846),
        ([-5000, -2000, 3000, 4000, 5000, 6000], 0.10, 5807.0114),
        ([-10000, 0, 0, 0, 0, 25000], 0.12, 4185.6714),
        ([-8000, -2000, 4000, 5000, 6000, 7000], 0.09, 6192.8460),
        ([-3000, 800, 800, 800, 800, 800], 0.06, 369.8910),
        ([-10000, 5000, -2000, 7000, 3000, 4000], 0.11, 2349.5975),
        ([10000, -3000, -3000, -3000, -3000], 0.07, -161.6338),
        ([-2000, 600, 600, 600, 600], 0.05, 127.5703),
        ([-15000, -5000, 4000, 6000, 8000, 10000], 0.13, -1799.7419),
        ([-5000, -1000, -500, 0, 8000, 9000], 0.15, 2800.9795),
        # 增加一些无解和多解的测试用例
        ([100, 200, 300], 0.05, 562.585034),  # 无 IRR
        ([-1000, 4000, -3000, 2000], 0.10, 1659.654395),  # 可能有多个 IRR
        ([-5000, 2000, 3000, -4000, 6000], 0.12, 143.283398),  # 可能有多个 IRR
        ([-2000, 5000, -4000, 3000, -1000], 0.08, 846.7412186),  # 可能有多个 IRR
    ]
    
    for cash_flows, discount_rate, expected_npv in test_cases:
        assert test_irr(cash_flows)
        assert test_npv(cash_flows, discount_rate, expected_npv)

    print("所有 IRR 和 NPV 测试通过！")
```

运行后的结果是：

```text
所有 IRR 和 NPV 测试通过！
```

### (3) 在主程序中使用新的 IRR 函数

我们回到现金流的主程序中，看看新的 IRR 运算在用户界面上的效果：

```text
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：1
----------------------------------------
----------------------------------------
请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：
请输入现金流列表：-100, 80, -30, 90, -20
现金流列表输入成功： [-100.0, 80.0, -30.0, 90.0, -20.0]
----------------------------------------
该现金流的内部收益率（IRR）为： [-0.7676970406112557, 0.1123585900902575]
请输入折现率（小数形式，如 0.05 表示 5%）：0.05
该现金流的净现值（NPV）为： 10.2709
----------------------------------------
是否继续计算？(y/n)：n
----------------------------------------
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：q
----------------------------------------
```

## 3. 查看新功能主函数中运行的效果

加入了两个新功能之后，我们来看看在主函数中运行的效果：

```text
========================================
                金融计算器
========================================
欢迎使用金融计算器！请选择以下功能：
0. 使用说明
1. 算式计算
2. 现金流量计算
3. 时间价值计算
l. 查看计算历史
q. 退出
----------------------------------------
请输入你的选择: 2
----------------------------------------
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：1
----------------------------------------
----------------------------------------
请输入现金流列表（以逗号分隔的数字，例如 -1000, 300, 400）：
请输入现金流列表：-1000, 200, 600, 800, 900, -500, -300, 400
现金流列表输入成功： [-1000.0, 200.0, 600.0, 800.0, 900.0, -500.0, -300.0, 400.0]
----------------------------------------
该现金流的内部收益率（IRR）为： [0.3285040514517912]
请输入折现率（小数形式，如 0.05 表示 5%）：0.06
该现金流的净现值（NPV）为： 788.1624
----------------------------------------
是否继续计算？(y/n)：n
----------------------------------------
----------------------------------------
现金流计算功能：
0. 使用说明
1. 输入现金流并进行运算
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：q
----------------------------------------
========================================
                金融计算器
========================================
欢迎使用金融计算器！请选择以下功能：
0. 使用说明
1. 算式计算
2. 现金流量计算
3. 时间价值计算
l. 查看计算历史
q. 退出
----------------------------------------
请输入你的选择: 3
----------------------------------------
----------------------------------------
金钱时间价值计算功能：
0. 查看功能说明
1. 计算金钱时间价值
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：1
----------------------------------------
----------------------------------------
请输入金钱时间价值的参数：
格式为：PV, FV, PMT, R, N，需要求的参数请输入 None
例如：-1000, None, -60, 0.05, 12
请输入参数：None, 2000, 60, 0.04, 20
金钱时间价值参数输入成功： [None, 2000.0, 60.0, 0.04, 20.0]
----------------------------------------
参数输入为：
PV： None
FV： 2000.0
PMT： 60.0
R： 0.04
N： 20.0
****************************************
计算结果为：
PV： -1728.193473100646
FV： 2000.0
PMT： 60.0
R： 0.04
N： 20.0
****************************************
输入任意内容返回主菜单：
----------------------------------------
----------------------------------------
金钱时间价值计算功能：
0. 查看功能说明
1. 计算金钱时间价值
q. 返回主菜单
----------------------------------------
请选择功能（0/1/q）：q
----------------------------------------
========================================
                金融计算器
========================================
欢迎使用金融计算器！请选择以下功能：
0. 使用说明
1. 算式计算
2. 现金流量计算
3. 时间价值计算
l. 查看计算历史
q. 退出
----------------------------------------
请输入你的选择: q
----------------------------------------
感谢使用，程序已退出！
```

可以看到，新加入的功能都整合到了主函数中，并且运行效果良好，用户体验也得到了提升。